edge cases not handled:
* merged contacts
* created date after closedate
* not sure if nightly job updates should count a day later, or count one day more

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
query = """
WITH temp_soft AS (
    SELECT sc.id
         , sc.npsp__opportunity__c
         , sc.npsp__contact__c
         , c.accountid
         , sc.effective_date__c
      FROM raw_salesforce.npsp__partial_soft_credit__c AS sc
      LEFT JOIN raw_salesforce.contact AS c ON c.id = sc.npsp__contact__c
     WHERE sc.npsp__role_name__c IN (
           'Workplace Giving Donor',
           'General',
           'Matching Donor',
           'Workplace Giving',
           'Donor Advised Fund',
           'Workplace  Giving',
           'Workplacegiving',
           'Matching Gift',
           'Workplace GIving'
           'Matching donor',
           'Matching Gift Donor',
           'Donor',
           'Matching Donors',
           'Maching Donor',
           'Donor Advised fund',
           'Donor Advise Fund')
   QUALIFY row_number() OVER (
           PARTITION BY npsp__opportunity__c ORDER BY effective_date__c, sc.createddate DESC
         ) = 1
)

SELECT
    CASE
        WHEN temp_soft.id IS NULL THEN o.accountid 
        ELSE temp_soft.accountid
    END AS account_id
    , o.closedate
    , SUM(o.amount) AS amount
 FROM raw_salesforce.opportunity AS o
 LEFT JOIN temp_soft ON o.id = temp_soft.npsp__opportunity__c
WHERE o.recordtypeid = '01241000000rfFPAAY'
  AND o.stagename = 'Posted'
  AND o.amount > 0
  AND DATE(o.closedate) < CURRENT_DATE  --ignore TODAY's donations since TODAY is incomplete
GROUP BY account_id, closedate
ORDER BY account_id, closedate
"""

In [3]:
import pandas_gbq
import pydata_google_auth
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']
credentials = pydata_google_auth.get_user_credentials(SCOPES, auth_local_webserver=True)

# Update the in-memory credentials cache (added in pandas-gbq 0.7.0).
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "u4u-ds-prod-00"

# The credentials and project_id arguments can be omitted.
df = pandas_gbq.read_gbq(query, dtypes={"closedate": "datetime64"})

Downloading: 100%|██████████|


In [24]:
# take sample of 100 account_ids
sample_account_ids = df['account_id'].drop_duplicates().sample(n=100, random_state=42)
df = df[df['account_id'].isin(sample_account_ids)]

In [35]:
# Given dataframe of [account_id,closedate, amount] function for checking if account has ever attained a status as defined by rolling sum of an amount over a time window
def membership_status_ever(df, minValue, maxValue, window):
    df['closedate'] = pd.to_datetime(df['closedate']).dt.tz_localize(None)
    # calcuate rolling sum along date
    amts = (
        df.groupby("account_id")
        .rolling(pd.Timedelta(days=window), on="closedate")["amount"]
        .sum()
        .reset_index()
    )
    # return dataframe with accout_id and boolean column indicating if amount ever between minValue and maxValue
    return (
        amts.groupby("account_id")["amount"]
        .apply(lambda x: ((x >= minValue) & (x <= maxValue)).any())
        .rename("ever_status")
        .reset_index()
    )


# Given dataframe of [account_id,closedate, amount] function for checking status changes as defined by rolling sum of an amount over a time window and between two dates
# Function still very slow so recommend limiting the number of accounts in the dataframe
def membership_status_change(df, minValue, maxValue, window, start_date, end_date):
    from tqdm import tqdm

    df['closedate'] = pd.to_datetime(df['closedate']).dt.tz_localize(None)

    results = pd.DataFrame(
        columns=[
            "account_id",
            "pre_status",
            "post_status",
            "downgraded_to_lower_status",
            "upgraded_to_higher_status",
            "downgraded_from_higher_status",
            "upgraded_to_status_or_new",
        ]
    )

    accounts = df["account_id"].unique()
    for account in tqdm(accounts):
        
        account_data = df[df["account_id"] == account]

        upgraded_to_higher_status = False
        upgraded_to_status_or_new = False
        downgraded_to_status = False
        downgraded_to_lower_status = False

        cumulative_pre = account_data[
            (account_data["closedate"] >= start_date - pd.Timedelta(days=window))
            & (account_data["closedate"] < start_date)
        ]["amount"].sum()

        if cumulative_pre < minValue:
            pre_status = -1
            status = -1
        elif cumulative_pre > maxValue:
            pre_status = 1
            status = -1
        else:
            pre_status = 0
            status = 0

        for date in pd.date_range(start_date, end_date):
            cumulative_up_to_date = account_data[
                (account_data["closedate"] >= date - pd.Timedelta(days=window))
                & (account_data["closedate"] <= date)
            ]["amount"].sum()
            if (status == 0) & (cumulative_up_to_date < minValue):
                status = -1
                downgraded_to_lower_status = True
            if (status == 0) & (cumulative_up_to_date > maxValue):
                status = 1
                upgraded_to_higher_status = True
            if (status == -1) & (
                (cumulative_up_to_date >= minValue)
                & (cumulative_up_to_date <= maxValue)
            ):
                status = 0
                upgraded_to_status_or_new = True
            if (status == 1) & (
                (cumulative_up_to_date >= minValue)
                & (cumulative_up_to_date <= maxValue)
            ):
                status = 0
                downgraded_to_status = True
        post_status = status

        results = pd.concat(
            [
                results,
                pd.DataFrame(
                    {
                        "account_id": [account],
                        "pre_status": [pre_status],
                        "post_status": [post_status],
                        "downgraded_to_lower_status": [downgraded_to_lower_status],
                        "upgraded_to_higher_status": [upgraded_to_higher_status],
                        "downgraded_to_status": [downgraded_to_status],
                        "upgraded_to_status_or_new": [upgraded_to_status_or_new],
                    }
                ),
            ]
        )
    # replace pre_status and post_status with human readable strings
    results["pre_status"] = results["pre_status"].replace(
        {0: "at_status", 1: "above_status", -1: "below_status_or_new"}
    )
    results["post_status"] = results["post_status"].replace(
        {0: "at_status", 1: "above_status", -1: "below_status_or_new"}
    )
    return results

In [36]:
start_date = pd.Timestamp("2021-01-01")
end_date = pd.Timestamp("2022-09-01")
window = 730  # 24 months
minValue = 1000  # this is min for mid-level
maxValue = 9999  # this is max for mid-level

# ever_midlevel = membership_status_ever(df, minValue, maxValue, window)

status_changes = membership_status_change(df, minValue, maxValue, window, start_date, end_date)

100%|██████████| 100/100 [00:26<00:00,  3.80it/s]


In [37]:
status_changes

,account_id,pre_status,post_status,downgraded_to_lower_status,upgraded_to_higher_status,downgraded_from_higher_status,upgraded_to_status_or_new,downgraded_to_status
0,0011K00002269wsQAA,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,0011K000023FqOnQAK,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,0011K000023GA28QAG,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,0011K000023GAbPQAW,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,0011K000023GB78QAG,below_status_or_new,below_status_or_new,False,False,NaN,False,False
...,...,...,...,...,...,...,...,...
0,001Rf000006qzGyIAI,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,001Rf000007tLcpIAE,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,001Rf000008Yr40IAC,below_status_or_new,below_status_or_new,False,False,NaN,False,False
0,001Rf00000HEmZFIA1,below_status_or_new,below_status_or_new,False,False,NaN,False,False
